In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import util

In [17]:
x,y = util.get_vecs()

## Keras

In [22]:
import keras
import sklearn
import sklearn.model_selection
import numpy as np

In [23]:
#from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)
x_train = np.array(x_train)
x_test = np.array(x_test)
print(x_train.shape)

(5658, 6654)


In [177]:
from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras import regularizers
model = Sequential()
#model.add(Dense(1024, activation="relu", input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1024, activation=LeakyReLU(alpha=0.1), input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
#model.add(Dense(512, activation="relu"))
model.add(Dense(512, activation=LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))
#model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation=LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 1024)              6814720   
_________________________________________________________________
dropout_29 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)               

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [ ]:
from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
from keras import regularizers
model = Sequential()
#model.add(Dense(1024, activation="relu", input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1024, activation=LeakyReLU(alpha=0.1), input_shape=(6654,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
#model.add(Dense(512, activation="relu"))
model.add(Dense(512, activation=LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))
#model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation=LeakyReLU(alpha=0.1)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

In [179]:
# Train the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(x=x_train, y=y_train, batch_size=100, epochs=100, verbose=True, validation_data=(x_test, y_test))

Train on 5658 samples, validate on 1886 samples
Epoch 1/100
5658/5658 [==============================] - 10s 2ms/step - loss: 1.7096 - accuracy: 0.5074 - val_loss: 1.4186 - val_accuracy: 0.5514
Epoch 2/100
5658/5658 [==============================] - 9s 2ms/step - loss: 1.2569 - accuracy: 0.6430 - val_loss: 1.2686 - val_accuracy: 0.5361
Epoch 3/100
5658/5658 [==============================] - 9s 2ms/step - loss: 1.0908 - accuracy: 0.7361 - val_loss: 1.3023 - val_accuracy: 0.5419
Epoch 4/100
5658/5658 [==============================] - 9s 2ms/step - loss: 0.9612 - accuracy: 0.8333 - val_loss: 1.5602 - val_accuracy: 0.5461
Epoch 5/100
5658/5658 [==============================] - 9s 2ms/step - loss: 0.9153 - accuracy: 0.8743 - val_loss: 1.7144 - val_accuracy: 0.5366
Epoch 6/100
5658/5658 [==============================] - 9s 2ms/step - loss: 0.8525 - accuracy: 0.9160 - val_loss: 1.9423 - val_accuracy: 0.5429
Epoch 7/100
5658/5658 [==============================] - 9s 2ms/step - loss: 0.82

KeyboardInterrupt: 

In [157]:
from sklearn import svm
from sklearn import metrics

In [159]:
svm_model = svm.SVC(kernel="rbf", verbose=True)
svm_model.fit(x_train, y_train)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [160]:
y_pred = svm_model.predict(x_test)
print("Accuracy: {}".format(metrics.accuracy_score(y_test, y_pred)))

Accuracy: 0.5355249204665959
